# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [99]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [100]:
# checking current working directory
print(os.getcwd())

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    # print(file_path_list)

E:\NutCloud\CodeVersion\CodeSample\Practice-Python\JupyterNoteBook\Udacity-数据工程师\【P2】Data Modeling with Cassandra


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [101]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile)  # just return an iterator
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# get total number of rows 
print(len(full_data_rows_list))

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                     'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

8056


In [102]:
# check the number of rows in csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

### The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [103]:
# make a connection to a Cassandra instance (127.0.0.1)
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# establish connection
session = cluster.connect()

#### Create Keyspace

In [104]:
# Create a Keyspace
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

#### Set Keyspace

In [105]:
# Set KEYSPACE to the keyspace created above
session.set_keyspace('udacity')

### Query 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

### create table

In [118]:
# create table for query 1
query = """
CREATE TABLE IF NOT EXISTS song_play_item_session
(
    session_id int,
    item_in_session int,
    artist_name text,
    song_name text,
    song_length float,
    primary key(session_id,item_in_session)
)
"""
session.execute(query)

### insert data

In [107]:
# read data from csv file and insert into the table above 
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "Insert Into song_play_item_session(session_id,item_in_session,artist_name,song_name,song_length)"
        query = query + "Values(%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]),int(line[3]),str(line[0]),str(line[9]),float(line[5])))

### validate data model

In [108]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
query = """
select artist_name,song_name,song_length
from song_play_item_session
where session_id = 338 and item_in_session = 4
"""
rows = session.execute(query)
df = pd.DataFrame(data = list(rows),columns=['artist_name','song_name','song_length'])
df.head()

,artist_name,song_name,song_length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


### Query 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

### create table

In [109]:
# create table for query 2
# use item_in_session as clusting key，then the data will sort by item_in_session
query = """
CREATE TABLE IF NOT EXISTS song_play_user_session
(
    user_id int,
    session_id int,
    item_in_session int,
    user_first_name text,
    user_last_name text,
    artist_name text,
    song_title text,
    primary key((user_id,session_id),item_in_session)
)
"""
session.execute(query)

### insert data

In [110]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "Insert Into song_play_user_session(user_id,session_id,item_in_session,user_first_name,\
                                                    user_last_name,artist_name,song_title)"
        query = query + "Values(%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]),int(line[8]),int(line[3]),str(line[1]),
                                str(line[4]),str(line[0]),str(line[9])))

### validate data model

In [113]:
query = """
select artist_name,song_title,user_first_name,user_last_name
from song_play_user_session
where user_id = 10 and session_id = 182
"""
rows = session.execute(query)
df = pd.DataFrame(data=list(rows),columns=['artist_name','song_title','user_first_name','user_last_name'])
df.head()

,artist_name,song_title,user_first_name,user_last_name
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


### Query 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

### create table

In [114]:
# create table for query 3
# add user_id to make primary key unique
query = """
CREATE TABLE IF NOT EXISTS song_play_user
(
    song_title text,
    user_id int,
    user_first_name text,
    user_last_name text,
    primary key(song_title,user_id)
)
"""
session.execute(query)

### insert data

In [115]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "Insert Into song_play_user(song_title,user_id,user_first_name,user_last_name)"
        query = query + "Values(%s, %s, %s, %s)"
        session.execute(query, (str(line[9]),int(line[10]),str(line[1]),str(line[4])))

### validate data model

In [117]:
query = """
select user_first_name,user_last_name
from song_play_user
where song_title = 'All Hands Against His Own'
"""
rows = session.execute(query)
df = pd.DataFrame(data=list(rows),columns=['user_first_name','user_last_name'])
df.head()

,user_first_name,user_last_name
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


### Drop the tables before closing out the sessions

In [96]:
session.execute('Drop Table If Exists song_play_item_session')
session.execute('Drop Table If Exists song_play_user_session')
session.execute('Drop Table If Exists song_play_user')

### Close the session and cluster connection¶

In [97]:
session.shutdown()
cluster.shutdown()